In [606]:
import pandas as pd
# pip install folium
import folium 

In [607]:
file_path = './data/2016_middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path)
df.head()

,Unnamed: 0,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,...,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도
0,0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,...,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909
1,1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,...,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857
2,2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,...,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744
3,3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,...,0.013,0.019,0.019,0.065,0.000,0.019,0,0.000,37.480439,127.062201
4,4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,...,0.010,0.005,0.000,0.282,0.000,0.010,0,0.000,37.510750,127.008900


In [608]:
school_map = folium.Map(location=[37.5, 127.032], zoom_start=15)

In [609]:
for name, latitude, longitude in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker(
        location=[latitude, longitude],
        radius=5,
        popup=name,
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(school_map)

In [610]:
school_map.save("./2016_middle.html")

In [611]:
df.head(3)

,Unnamed: 0,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,...,외고_국제고,예고_체고,마이스터고,자사고,자공고,기타진학,취업,미상,위도,경도
0,0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,...,0.007,0.000,0.011,0.227,0.000,0.004,0,0.000,37.594942,127.038909
1,1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,...,0.035,0.008,0.000,0.043,0.004,0.031,0,0.000,37.577473,127.003857
2,2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,...,0.012,0.003,0.006,0.090,0.003,0.009,0,0.003,37.491637,127.071744


## Preprocessing

In [612]:
from sklearn import preprocessing

In [613]:
label_encoder = preprocessing.LabelEncoder()

In [614]:
# 지역, 코드 유형 주야
label_location = label_encoder.fit_transform(df['지역'])
label_code =  label_encoder.fit_transform(df['코드'])
label_type = label_encoder.fit_transform(df['유형'])
label_day = label_encoder.fit_transform(df['주야'])

In [615]:
df['location'] = label_location
df['code'] = label_code
df['type'] = label_type
df['day'] = label_day

In [616]:
df.drop(inplace=True,columns='Unnamed: 0')
df.head(3)

,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,과학고,...,자공고,기타진학,취업,미상,위도,경도,location,code,type,day
0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,0.018,...,0.000,0.004,0,0.000,37.594942,127.038909,16,0,1,0
1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,0.000,...,0.004,0.031,0,0.000,37.577473,127.003857,22,0,1,0
2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,0.009,...,0.003,0.009,0,0.003,37.491637,127.071744,0,0,0,0


In [617]:
x = df.iloc[:,[9,10,13]]  # 과학고, 외고(국제고), 자사고
x.head()

,과학고,외고_국제고,자사고
0,0.018,0.007,0.227
1,0.000,0.035,0.043
2,0.009,0.012,0.090
3,0.013,0.013,0.065
4,0.007,0.010,0.282


In [618]:
x = preprocessing.StandardScaler().fit(x).transform(x)

In [619]:
# DBSCAN 모델링 => cluster

In [620]:
from sklearn import cluster

In [621]:
dbscan = cluster.DBSCAN(eps=0.3, min_samples=4) 
dbscan.fit(x)  #학습

DBSCAN(eps=0.3, min_samples=4)

In [622]:
df['Cluster'] = dbscan.labels_
group_col = [0,1,3,9,10,13]  # 지역, 학교명, 유형, 과학고, 외고, 자사고

In [623]:
grouped = df.groupby('Cluster')

In [624]:
for key, group in grouped:
    print(key, len(group))
    #print(group.iloc[:,group_col].head(10))
    print("과학고 : ",(group.과학고.sum()/len(group)).round(5) * 100)
    print("외고_국제고 : ",(group.외고_국제고.sum()/len(group)).round(5) * 100)
    print("자사고 : ",(group.자사고.sum()/len(group)).round(5) * 100)
    total = group.과학고.sum() + group.외고_국제고.sum() + group.자사고.sum()
    print("전체 특목고 : ",(total/len(group)).round(5) * 100)
    print('\n')


-1 161
과학고 :  0.893
외고_국제고 :  2.0140000000000002
자사고 :  13.352
전체 특목고 :  16.258


0 172
과학고 :  0.0
외고_국제고 :  0.8869999999999999
자사고 :  3.73
전체 특목고 :  4.617


1 12
과학고 :  0.4
외고_국제고 :  1.417
자사고 :  7.008
전체 특목고 :  8.825


2 4
과학고 :  0.375
외고_국제고 :  2.1
자사고 :  6.7250000000000005
전체 특목고 :  9.2


3 6
과학고 :  0.65
외고_국제고 :  2.033
자사고 :  11.05
전체 특목고 :  13.733


4 4
과학고 :  0.975
외고_국제고 :  1.7999999999999998
자사고 :  6.375
전체 특목고 :  9.15


5 21
과학고 :  0.45199999999999996
외고_국제고 :  1.171
자사고 :  2.524
전체 특목고 :  4.148000000000001


6 4
과학고 :  0.7000000000000001
외고_국제고 :  1.275
자사고 :  6.800000000000001
전체 특목고 :  8.774999999999999


7 5
과학고 :  0.0
외고_국제고 :  0.76
자사고 :  14.44
전체 특목고 :  15.2


8 18
과학고 :  0.383
외고_국제고 :  0.35000000000000003
자사고 :  5.311
전체 특목고 :  6.0440000000000005


9 4
과학고 :  0.575
외고_국제고 :  0.0
자사고 :  10.6
전체 특목고 :  11.175


10 4
과학고 :  0.6
외고_국제고 :  1.6500000000000001
자사고 :  18.95
전체 특목고 :  21.2




In [625]:
# 지도에 표시
cluster_map = folium.Map(location=[37.5, 127.032], zoom_start=12)
cluster_colors = {-1:'black',0:'green',1 :'blue',2:'yellow',3:'purple',4:'brown',5:'red', 6:'skyblue', 7:'gray',8:'pink',9:'gold',10:'black'}

In [626]:
for name, latitude, longitude, cluster in zip(df.학교명, df.위도, df.경도, df.Cluster):
    folium.CircleMarker(
        location=[latitude, longitude],
        radius=5,
        popup=name,
        color=cluster_colors[cluster],
        fill=True,
        fill_color=cluster_colors[cluster]
    ).add_to(cluster_map)
cluster_map.save("./2016_middle_cluster.html")

In [627]:
x2 = df.iloc[:,[9,10]]  # 과학고, 외고(국제고), 자사고
x2.head()

,과학고,외고_국제고
0,0.018,0.007
1,0.000,0.035
2,0.009,0.012
3,0.013,0.013
4,0.007,0.010


In [628]:
x2 = preprocessing.StandardScaler().fit(x2).transform(x2)

In [629]:
from sklearn import cluster
dbscan2 = cluster.DBSCAN(eps=0.3, min_samples=5) 
dbscan2.fit(x)  #학습
df['Cluster2'] = dbscan2.labels_
df.head()

,지역,학교명,코드,유형,주야,남학생수,여학생수,일반고,특성화고,과학고,...,취업,미상,위도,경도,location,code,type,day,Cluster,Cluster2
0,성북구,서울대학교사범대학부설중학교,3,국립,주간,277,0,0.585,0.148,0.018,...,0,0.000,37.594942,127.038909,16,0,1,0,-1,-1
1,종로구,서울대학교사범대학부설여자중학교,3,국립,주간,0,256,0.680,0.199,0.000,...,0,0.000,37.577473,127.003857,22,0,1,0,-1,-1
2,강남구,개원중학교,3,공립,주간,170,152,0.817,0.047,0.009,...,0,0.003,37.491637,127.071744,0,0,0,0,-1,-1
3,강남구,개포중학교,3,공립,주간,83,72,0.755,0.097,0.013,...,0,0.000,37.480439,127.062201,0,0,0,0,-1,-1
4,서초구,경원중학교,3,공립,주간,199,212,0.669,0.017,0.007,...,0,0.000,37.510750,127.008900,14,0,0,0,-1,-1


In [630]:
group2_col = [0,1,3,9,10]
group2 = df.groupby('Cluster2')
for key, group in group2:
    print(key, len(group))
    print("과학고 : ",(group.과학고.sum()/len(group)).round(5) * 100)
    print("외고_국제고 : ",(group.외고_국제고.sum()/len(group)).round(5) * 100)
    total = group.과학고.sum() + group.외고_국제고.sum()
    print("과고+외고 : ",(total/len(group)).round(5) * 100)
    print('\n')

-1 197
과학고 :  0.819
외고_국제고 :  1.8429999999999997
과고+외고 :  2.662


0 171
과학고 :  0.0
외고_국제고 :  0.8920000000000001
과고+외고 :  0.8920000000000001


1 12
과학고 :  0.4
외고_국제고 :  1.417
과고+외고 :  1.817


2 6
과학고 :  0.65
외고_국제고 :  2.033
과고+외고 :  2.683


3 11
과학고 :  0.42700000000000005
외고_국제고 :  0.964
과고+외고 :  1.391


4 13
과학고 :  0.385
외고_국제고 :  0.377
과고+외고 :  0.762


5 5
과학고 :  0.38
외고_국제고 :  1.52
과고+외고 :  1.9




In [631]:
cluster2_map = folium.Map(location=[37.5, 127.032], zoom_start=12)
for name, latitude, longitude, cluster in zip(df.학교명, df.위도, df.경도, df.Cluster2):
    folium.CircleMarker(
        location=[latitude, longitude],
        radius=5,
        popup=name,
        color=cluster_colors[cluster],
        fill=True,
        fill_color=cluster_colors[cluster]
    ).add_to(cluster2_map)
cluster2_map.save("./2016_middle_cluster2.html")